## OCR Report
This notebook constructs a proof-of-concept report of the district level data of Washington State schools, describing discipline and attendance information and highlighting disproportionate areas. See the README [here](https://github.com/CBofill/OCR-Discipline-and-Attendance-Visualization-Tool) for more information.

In [1]:
%matplotlib inline

import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt

In [2]:
input_file = './data/NeedDistrict_Part2.csv'
output_file = './calculated_data.csv'

In [3]:
# Import the data
df = pd.read_csv(input_file,index_col='SCH_ID')

In [4]:
df.replace('Yes', True, inplace=True)
df.replace('No', False, inplace=True)

In [5]:
## get list of grades
grades = [x.strip() for x in re.findall('SCH_GRADE_...', ' '.join(df.columns))]

## create variable for grades in a given school
school_grade_span = []
for row in df.iterrows():
    span = [re.sub('SCH_GRADE_', '', g) for g in grades if row[1][g]]
    school_grade_span.append((span[0], span[-1]))
## add grade span to data set
df['school_grade_span'] = pd.Series(school_grade_span, index=df.index)

## Enrollment
Enrollment is provided by race/ethnicity separated by gender, as an inclusive count of all students in the school. The number of these students who have disabilities (IDEA/504) is also provided by race/ethnicity.

The total for each race/ethnicity is calculated, as well as the total students enrolled in each school. This is repeated for students with disabilities.

Abbreviation-race pairs:

    - HI: Hispanic
    - AM: American Indian/ Alaksan Native
    - AS: Asian
    - HP: Native Hawaiian/Pacific Islander
    - BL: Black
    - WH: White
    - TR: Two or more

In [6]:
## get list of columns with enrollment data
enrollment_columns = [x for x in re.findall('SCH_ENR_.._[MF]', ' '.join(df.columns))]
enrollment_columns.extend(['school_grade_span'])

#get just enrollment data
enrollment_data = df.loc[:,enrollment_columns] 

#clean up names of columns
enrollment_data.columns = [re.sub('SCH_ENR_', '', x) for x in enrollment_data.columns]

# add columns representing all male and female members of given race
race_abbreviations = ['HI', 'AM', 'AS', 'HP', 'BL', 'WH', 'TR']
for r in race_abbreviations:
    enrollment_data[r] = enrollment_data[re.findall(r + '_.', ' '.join(enrollment_data.columns))].sum(axis=1)

# add column for male and female
gender = ['M', 'F']
for g in gender:
    enrollment_data[g] = enrollment_data[re.findall( r'\b.._' + g, ' '.join(enrollment_data.columns))].sum(axis=1)

# create df for presenting data
enrollment_data_clean = enrollment_data.ix[:,-9:]
enrollment_data_clean['total'] = enrollment_data_clean.iloc[:,-2:].sum(axis=1)

In [7]:
enrollment_data_clean

,HI,AM,AS,HP,BL,WH,TR,M,F,total
SCH_ID,,,,,,,,,,
530003000005,28,2,3,0,2,109,14,80,78,158
530003000010,134,10,1,1,2,183,26,168,189,357
530003000008,109,5,4,0,3,210,29,176,184,360
530003000011,185,25,16,0,9,248,37,274,246,520
530003000001,180,13,8,0,3,198,17,214,205,419
530003000009,119,30,13,2,6,268,18,218,238,456
530003000007,257,33,30,6,8,548,39,477,444,921
530003000002,25,5,1,1,4,62,8,58,48,106
530003002055,9,1,0,0,1,12,0,14,9,23


In [8]:
# repeat above for students with disabilities
enrollment_wdis_columns = [x for x in re.findall('SCH_(?:IDEA|504)ENR_.._[MF]', ' '.join(df.columns))]
enrollment_wdis_data = df.loc[:,enrollment_wdis_columns]
enrollment_wdis_data.columns = [re.sub('SCH_(?:IDEA|504)ENR_', '', x) for x in enrollment_wdis_data.columns]
for r in race_abbreviations:
    enrollment_wdis_data[r] = enrollment_wdis_data[re.findall(r + '_.', ' '.join(enrollment_data.columns))].sum(axis=1)
for g in gender:
    enrollment_wdis_data[g] = enrollment_wdis_data[re.findall(r'\b.._' + g, ' '.join(enrollment_data.columns))].sum(axis=1)

enrollment_wdis_clean = enrollment_wdis_data.ix[:,-9:]
enrollment_wdis_clean['total'] = enrollment_wdis_data.iloc[:,-2:].sum(axis=1)

In [9]:
enrollment_wdis_clean

,HI,AM,AS,HP,BL,WH,TR,M,F,total
SCH_ID,,,,,,,,,,
530003000005,4.0,1.0,0.0,0.0,0.0,9.0,4.0,12.0,6.0,18.0
530003000010,14.0,1.0,0.0,0.0,0.0,21.0,2.0,21.0,17.0,38.0
530003000008,15.0,2.0,2.0,0.0,1.0,23.0,5.0,33.0,15.0,48.0
530003000011,26.0,5.0,0.0,0.0,3.0,47.0,6.0,55.0,32.0,87.0
530003000001,31.0,6.0,1.0,0.0,0.0,37.0,5.0,50.0,30.0,80.0
530003000009,11.0,3.0,2.0,0.0,0.0,44.0,3.0,39.0,24.0,63.0
530003000007,47.0,11.0,3.0,1.0,1.0,99.0,8.0,103.0,67.0,170.0
530003000002,5.0,1.0,1.0,0.0,1.0,16.0,1.0,15.0,10.0,25.0
530003002055,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [10]:
# create df of students without disabilities
enrollment_wodis = enrollment_data_clean - enrollment_wdis_clean

In [11]:
enrollment_wodis

,HI,AM,AS,HP,BL,WH,TR,M,F,total
SCH_ID,,,,,,,,,,
530003000005,24.0,1.0,3.0,0.0,2.0,100.0,10.0,68.0,72.0,140.0
530003000010,120.0,9.0,1.0,1.0,2.0,162.0,24.0,147.0,172.0,319.0
530003000008,94.0,3.0,2.0,0.0,2.0,187.0,24.0,143.0,169.0,312.0
530003000011,159.0,20.0,16.0,0.0,6.0,201.0,31.0,219.0,214.0,433.0
530003000001,149.0,7.0,7.0,0.0,3.0,161.0,12.0,164.0,175.0,339.0
530003000009,108.0,27.0,11.0,2.0,6.0,224.0,15.0,179.0,214.0,393.0
530003000007,210.0,22.0,27.0,5.0,7.0,449.0,31.0,374.0,377.0,751.0
530003000002,20.0,4.0,0.0,1.0,3.0,46.0,7.0,43.0,38.0,81.0
530003002055,8.0,1.0,0.0,0.0,1.0,12.0,0.0,13.0,9.0,22.0


In [12]:
# Function to calculate data proportions based off values in 'total'. Returns df of same shape as input.
def calculate_proportions(df):
    data_proportions = df.copy()
    if 'total' not in data_proportions.columns:
        raise ValueError('Input must have \'total\' column')
    for column in data_proportions.columns:
        data_proportions[column] = data_proportions[column]*1.0 / data_proportions['total']
    return data_proportions

In [13]:
# get proportions for total enrollment, and students with and without disabilities
enrollment_proportions = calculate_proportions(enrollment_data_clean)
enrollment_wdis_proportions = calculate_proportions(enrollment_wdis_clean)
enrollment_wodis_proportions = calculate_proportions(enrollment_wodis)

In [14]:
# create df for presenting enrollment data
enrollment_presentation = enrollment_proportions.round(decimals=2)
enrollment_presentation.drop('total', axis=1, inplace=True)
enrollment_presentation

,HI,AM,AS,HP,BL,WH,TR,M,F
SCH_ID,,,,,,,,,
530003000005,0.18,0.01,0.02,0.00,0.01,0.69,0.09,0.51,0.49
530003000010,0.38,0.03,0.00,0.00,0.01,0.51,0.07,0.47,0.53
530003000008,0.30,0.01,0.01,0.00,0.01,0.58,0.08,0.49,0.51
530003000011,0.36,0.05,0.03,0.00,0.02,0.48,0.07,0.53,0.47
530003000001,0.43,0.03,0.02,0.00,0.01,0.47,0.04,0.51,0.49
530003000009,0.26,0.07,0.03,0.00,0.01,0.59,0.04,0.48,0.52
530003000007,0.28,0.04,0.03,0.01,0.01,0.60,0.04,0.52,0.48
530003000002,0.24,0.05,0.01,0.01,0.04,0.58,0.08,0.55,0.45
530003002055,0.39,0.04,0.00,0.00,0.04,0.52,0.00,0.61,0.39


## In-School Suspensions
In-school suspension data are provided by race/ethnicity separated by gender and students with and without disabilites. 

The percentage rate of students who received one or more in-school suspension is summed by race/ethnicity, then the composition index is calculated. Students with and without disabilities are considered separately.

In [ ]:
## get list of columns with ISS data
iss_columns = [x for x in re.findall('SCH_DISCWODIS_ISS_.._[MF]', ' '.join(df.columns))]
iss_columns.extend(['school_grade_span'])

#get just iss data
iss_df = df.loc[:,iss_columns]
#clean up names of columns
iss_df.columns = [re.sub('SCH_DISCWODIS_ISS_', '', x) for x in iss_df.columns]
#iss_df = iss_df.groupby(by = iss_df.columns, axis=1).sum()

# add columns representing all male and female members of given race
for r in race_abbreviations:
    iss_df[r] = iss_df[re.findall(r + '_.', ' '.join(iss_df.columns))].sum(axis=1)

# add column for male and female
gender = ['M', 'F']
for g in gender:
    iss_df[g] = iss_df[re.findall( '.._' + g, ' '.join(iss_df.columns))].sum(axis=1)
    
#iss_dis_columns = [x for x in re.findall('SCH_DISCWDIS_ISS_(?:IDEA|504)_.._[MF]', ' '.join(df.columns))]
#iss_df.columns = [re.sub('SCH_DISCWDIS_ISS_(?:IDEA|504)_', '', x) for x in iss_df.columns]

In [ ]:
# get clean iss data
iss_df_clean = iss_df.iloc[:, -9:]
iss_df_clean['total'] = iss_df_clean.iloc[:,-2:].sum(axis=1)
iss_df_clean

In [ ]:
# get iss proportions
iss_proportions = iss_df_clean.copy()
for i in xrange(iss_proportions.shape[0]):
    for j in xrange(iss_proportions.shape[1]):
        if iss_proportions.iloc[i, -1] != 0:
            iss_proportions.iloc[i, j] = iss_proportions.iloc[i, j]*1.0 / iss_proportions.iloc[i, -1]

In [ ]:
# create df for displaying iss data
iss_presentation = iss_proportions.round(decimals=2)
iss_presentation.drop('total', axis=1, inplace=True)
iss_presentation

## Out-of-School Suspensions
Out-of-school suspension data are provided by race/ethnicity separated by gender, students with and without disabilities, and students receiving only one or more than one suspension. The percentage rate of students who received one or more out-of-school suspension is summed by race/ethnicity, and the composition index is calculated.

In [ ]:
"""
## get list of columns with OOS suspension data
oos_columns = [x for x in re.findall('SCH_DISCWODIS_(MULT|SING)OOS_.._(M|F)', ' '.join(df.columns))]
oos_columns.extend([x for x in re.findall('SCH_DISCWDIS_(MULT|SING)OOS_IDEA_.._(M|F)', ' '.join(df.columns))])
oos_columns.extend(['school_grade_span'])

#get just oos data
oos_df = df[oos_columns]
#clean up names of columns
oos_df.columns = [re.sub('SCH_DISCWODIS_....OOS_', '', x) for x in oos_df.columns]
oos_df.columns = [re.sub('SCH_DISCWDIS_....OOS_IDEA_', '', x) for x in oos_df.columns]
oos_df = oos_df.groupby(by = oos_df.columns, axis=1).sum()

# add columns representing all male and female members of given race
for r in race_abbreviations:
    oos_df[r] = oos_df[re.findall(r + '_.', ' '.join(oos_df.columns))].sum(axis=1)

# add column for male and female
gender = ['M', 'F']
for g in gender:
    oos_df[g] = oos_df[re.findall( '.._' + g, ' '.join(oos_df.columns))].sum(axis=1)
""";

## Expulsions
Expulsions data are provided by race/ethnicity separated by gender, students with and without disabilities, and students who received an expulsion with or without educational services. The percentage rate of students who received an expulsion is summed by race/ethnicity, and the composition index is calculated.

In [ ]:
"""# Calculate expulsion totals
expulsion_data = pd.DataFrame()

for race in race_abbreviations:
    expulsion_data[race] = data[['SCH_PSDISC_EXP_'+race+'_M','SCH_PSDISC_EXP_'+race+'_F',
                                            'SCH_DISCWODIS_EXPWE_'+race+'_M','SCH_DISCWODIS_EXPWE_'+race+'_F',
                                            'SCH_DISCWODIS_EXPWOE_'+race+'_M','SCH_DISCWODIS_EXPWOE_'+race+'_F',
                                            'SCH_DISCWDIS_EXPWE_IDEA_'+race+'_M','SCH_DISCWDIS_EXPWE_IDEA_'+race+'_F',
                                            'SCH_DISCWDIS_EXPWOE_IDEA_'+race+'_M',
                                             'SCH_DISCWDIS_EXPWOE_IDEA_'+race+'_F']].sum(axis=1)
expulsion_data['total'] = expulsion_data.sum(axis=1)
    
# Calculate expulsion percentages
for race in race_abbreviations:
    expulsion_data[race+'_p'] = expulsion_data[race]/expulsion_data['total']
    
# Calculate composition index
for race in race_abbreviations:
    expulsion_data[race+'_comp'] = expulsion_data[race+'_p']/enrollment_data[race+'_p']
    """

## Visualization

In [ ]:
iss_proportions

In [ ]:
enrollment_data_proportions

In [ ]:
school_enrollment = enrollment_data_proportions.iloc[6,:-3].values
school_iss = iss_proportions.iloc[6,:-3].values
indices = range(len(race_abbreviations))
width = 0.5

composition = school_iss / school_enrollment

flagging = []
for r in composition:
    if r > 1.0:
        flagging.extend('r')
    else:
        flagging.extend('b')

plt.bar(indices, school_enrollment, color='c', alpha = 0.5, width=width, label='Enrollment')
plt.bar([i+0.25*width for i in indices], school_iss, color=flagging, width = 0.5*width, alpha=0.5, label='ISS')


plt.xticks([i+0.5*width for i in indices], race_abbreviations)
plt.title('School {}: Enrollment v. ISS'.format(enrollment_data_clean.index[6]))
plt.ylim(0, 0.9)
plt.legend()
plt.show()